In [1]:
# python warning off
import warnings
warnings.filterwarnings('ignore')

# tensorflow warning off
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
import pandas as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
def read_ver1(what, how):
#원하는 데이터 연도별로 다 불러오가
#what은 무슨 기록인지 받는 입력   예)팀타자, 개인투수....
#how는 생성할 데이터 프레임 이름  예)df_raw, df_bat...
    year = [16,17,18,19,20]
    for i in year:
        globals()['{0}{1}'.format(how,i)] = pd.read_csv("datasets/2020빅콘테스트_스포츠투아이_제공데이터_{0}_20{1}.csv".format(what,i))

In [4]:
read_ver1('개인투수','pit')

In [5]:
def teams(df):
#팀코드(T_ID) 리스트 생성
    team = list(set(df['T_ID'].values))
    return team

In [6]:
team = teams(pit16)

In [7]:
def gen_pit(df,year):
    #개인투수 csv를 넣으면 
    #팀별 전체 데이터 프레임 = '{0}_all'
    #팀별 선발 데이터 프레임 ='{0}_start'
    #팀별 선발 횟수 데이터 프레임 = '{0}_count'
    #생성

    team = list(set(df['T_ID'].values))
    # 팀 리스트 생성, 한경기에 등판한 투수가 모두 있기에 set 자료형으로 중복을 없애고 리스트 변환한다.
    
    for i in team:
        globals()['{0}_all{1}'.format(i,year)] = df[df['T_ID']==i]
    for i in team:
        globals()['{0}_start{1}'.format(i,year)] = globals()['{0}_all{1}'.format(i, year)][globals()['{0}_all{1}'.format(i,
                                                                                                                year)]['START_CK']==1]
    for i in team:
        globals()['{0}_count{1}'.format(i, year)]=pd.DataFrame(globals()['{0}_start{1}'.format(i,
                                                                                        year)]['P_ID'].value_counts()).reset_index(
        ).rename(columns={'P_ID':'선발 횟수','index':'P_ID'})

In [8]:
year = [16,17,18,19,20]
for i in year:
    gen_pit(globals()['pit{0}'.format(i)], i)

In [9]:
def gen_data(year):
    for i in team:#team에 대한 정의는 위 teams 함수의 반환값임.
        globals()['data_{0}{1}'.format(i,year)] =  globals()['{0}_start{1}'.format(i, year)].loc[:,('GDAY_DS','P_ID')]


In [10]:
year = [16,17,18,19,20]
for i in year:
    gen_data(i)

In [11]:
def ID_label(a):
    ab = a['P_ID'].values
    abc = []
    for i in ab:
        if i in abc:
            pass
        else:
            abc.append(i)
    abcd = []
    for i in ab:
        abcd.append(abc.index(i))
    a["P_ID"] =  abcd
    
    e = {}
    for i in abc:
        e[i]= abc.index(i)
    print(e)

In [12]:
def date(df):
    #일자 컬럼 추가
    ser_date = pd.to_datetime(df['GDAY_DS'], format='%Y%m%d',errors='ignore')
    df['date']=ser_date
    df = df.reset_index().drop(['index','GDAY_DS'], axis=1)
    df['월'] = df['date'].dt.month
    df['일'] = df['date'].dt.day
    return df

In [13]:
for i in team:
    for j in year:
        globals()['data_{0}{1}'.format(i,j)] = date(globals()['data_{0}{1}'.format(i,j)])
        ID_label(globals()['data_{0}{1}'.format(i,j)])

{62698: 0, 73117: 1, 63111: 2, 62463: 3, 61101: 4, 66154: 5, 77147: 6, 61145: 7, 74139: 8, 60164: 9, 66138: 10, 66160: 11}
{62698: 0, 63111: 1, 61329: 2, 76455: 3, 61101: 4, 66145: 5, 66138: 6}
{68135: 0, 66145: 1, 62698: 2, 61101: 3, 64155: 4, 76455: 5, 67143: 6, 68130: 7, 60140: 8, 79825: 9, 63145: 10}
{68135: 0, 69103: 1, 61101: 2, 63145: 3, 76455: 4, 66145: 5, 76355: 6, 61145: 7, 63111: 8, 60140: 9, 74167: 10, 69158: 11}
{76455: 0, 73801: 1, 78148: 2, 68135: 3, 69103: 4, 61101: 5, 50126: 6, 50157: 7}
{63938: 0, 65931: 1, 61323: 2, 62929: 3, 60263: 4, 66960: 5, 64995: 6, 66990: 7, 64960: 8, 62966: 9, 65933: 10, 63950: 11}
{67948: 0, 60263: 1, 65933: 2, 62966: 3, 63938: 4, 63950: 5, 64995: 6, 62929: 7, 79358: 8, 66960: 9, 62951: 10}
{68948: 0, 68953: 1, 62966: 2, 65933: 3, 60263: 4, 66960: 5, 60339: 6, 66920: 7, 62920: 8, 62951: 9, 68908: 10, 63991: 11}
{69934: 0, 69940: 1, 60263: 2, 68900: 3, 63991: 4, 66920: 5, 69962: 6, 76757: 7, 65933: 8, 63936: 9, 69953: 10, 66960: 11}
{69940: 0

In [14]:
def VS_team(org,new):
    org['VS_T_ID'].reset_index()['VS_T_ID']
    new['VS_T_ID'] = org['VS_T_ID'].reset_index()['VS_T_ID']
    from sklearn.preprocessing import LabelEncoder
    label = LabelEncoder()
    new['VS_T_ID'] = label.fit_transform(new['VS_T_ID'])

In [15]:
for i in team:
    for j in year:
        VS_team(globals()['{0}_start{1}'.format(i,j)],globals()['data_{0}{1}'.format(i,j)])

In [16]:
def p_fac(df , new):
    from sklearn.preprocessing import LabelEncoder
    label = LabelEncoder()
    a = []
    for i in df['G_ID']:
        a.append(i[8:10])
    new['p_fac'] = a
    new['p_fac'] = label.fit_transform(new['p_fac'])

In [17]:
for i in team:
    for j in year:
        p_fac(globals()['{0}_start{1}'.format(i,j)],globals()['data_{0}{1}'.format(i,j)])

In [18]:
for i in team:
    for j in year:
        globals()['data_{0}{1}'.format(i,j)].to_csv('pred_start/data_{0}{1}.csv'.format(i,j), index =False)

In [19]:
def read_ver2(what, how):
#원하는 데이터 연도별로 다 불러오가
#what은 무슨 기록인지 받는 입력   예)팀타자, 개인투수....
#how는 생성할 데이터 프레임 이름  예)df_raw, df_bat...
    year = [16,17,18,19,20]
    for i in year:
        globals()['{0}{1}'.format(how,i)] = pd.read_csv("pred_start/data_{0}{1}.csv".format(what,i))

In [20]:
for i in team:
    read_ver2(i,i)

In [21]:
def mod_label(mod_label, org, new):
    if org not in mod_label.keys():
        mod_label[org]= [new]
    else:
        if new not in mod_label[org]:
            b = mod_label[org]
            b.append(new)
            mod_label[org] = b

In [22]:
def num_under(df):
    b = list(df['P_ID'].value_counts().index)
    c=[]
    d = 0
    for i in b:
        if df['P_ID'].value_counts()[i] >= 20:
            d +=1
            pass
        else:
            c.append(i)
    e = c[:5-d] # 이는 축이 될 번호들
    c = c[5-d:]
    c.reverse() #등판 수가 가장 적은 것이 앞에 있는 투수 ID 리스트 생성
    return e, c

def devide(i,df,pitlist):
    res =[]
    c = []
    first = df[df['P_ID']==i].index
    for k in first:
        c.append(list(df['P_ID'][k:k+5].values))
    for j in pitlist:
        res.append(j)
        for l in c:
            if j not in l:
                continue
            elif j in l:
                res.remove(j)
                break
    return res


def rule3_1(df,group):
    a =0
    for i in group:
        a += df['P_ID'].value_counts()[i]
    return a


def rule3_2(i, j,df):
    if df['P_ID'].value_counts()[i] + j > 29:
        return 'over'
    else:
        return 'under'
    

def label_group(group, df,label):
    for i in group:
        testi = list(df[df['P_ID']==i]['P_ID'].index)
        df['P_ID'][testi] = label
        mod_label(mody_label, i, label)

def add_label(df,add_list):
    label_list = set(df['P_ID'].value_counts().index) - add_list #이 라벨 리스트는 축이 될 라벨들
    res ={}
    c = []
    value_list =[]
    
    for i in label_list:
        if df['P_ID'].value_counts()[i] >= 29: #33이상 등판인 투수는 축에서 제외한다.
            continue
        
        first = df[df['P_ID']==i].index 
        value_list.append(i)
        for k in first:
            if k == 0:
                c.append(list(df['P_ID'][0:k+1].values))
            else:
                c.append(list(df['P_ID'][k-1:k+1].values))
        # 앞뒤 등판 여부 판단하기 위해 c리스트 생성
        for j in add_list:     #j는 변경되어야 할 라벨
            v_list = value_list[:]
            res[j] = v_list
            for l in c:
                if j not in l:
                    continue
                elif j in l:
                    v_list.remove(i)
                    res[j] = v_list
                    break
        #만약 i(축이 될 라벨)에 앞 뒤에 등판 한적이 없다면 j를 키로 value에 i를 추가
        
#         label_group(res.keys(), df, list(res.values())[0]) #여기 문제가 많다! 
#         num2 - set(label_group)
    
    return res

# def label_group(group, df,label):
#     for i in group:
#         testi = list(df[df['P_ID']==i]['P_ID'].index)
#         df['P_ID'][testi] = label


#이 다음은 아직 함수 정의를 하지 않은 것이다.
# b = list(a.keys())
# c = list(a.values())
# for i in range(len(a)):
#     for j in range(len(c[i])):
#        if df['P_ID'].value_counts()[b[i]] +df["P_ID"].value_counts()[c[i][j]] <= 29:
#            testi = list(df[df["P_ID"]==b[i]]["P_ID"].index)
#            df['P_ID'][testi] = c[i][j]  #코드를 완성하고 주석을 해제하자.
#            break   #break는 반복문을 빠져나온다... 조건문이 아니라..
def rule5(a,df):
    b = list(a.keys())
    c = list(a.values())
    for i in range(len(a)):
        for j in range(len(c[i])):
            if df['P_ID'].value_counts()[b[i]] +df["P_ID"].value_counts()[c[i][j]] <= 29:
                testi = list(df[df["P_ID"]==b[i]]["P_ID"].index)
                df['P_ID'][testi] = c[i][j] 
                mod_label(mody_label,b[i] , c[i][j])
                break


def rule6(df):
    label = df['P_ID'].value_counts().index[:5]
    extra = df['P_ID'].value_counts().index[5:]
    for i in label:
        for j in extra:
            if df['P_ID'].value_counts()[j] +df["P_ID"].value_counts()[i] <= 29:
                testi = list(df[df["P_ID"]==j]["P_ID"].index)
                df['P_ID'][testi] = i
                mod_label(mody_label,j , i)
                
def rule7(df):
    index = list(df['P_ID'].value_counts().index)
    obj = []
    obj_num = []
    input = index[5:]
    value = index[:5]
    for i in value:
        if df['P_ID'].value_counts()[i] < 29:
            obj.append(i)
            obj_num.append(29-df['P_ID'].value_counts()[i])
    for i in input:
        testi = list(df[df['P_ID']==i]['P_ID'].index) #지금 테스트i 까지 바꾸었다..
            
    return_list = []
    for c in input:
        d=0
        b =  list(df[df["P_ID"]==c].index)
        for i in range(len(obj_num)):
            df["P_ID"][b[d:d+obj_num[i]]] = obj[i]
            mod_label(mody_label, c , obj[i])
            d+=obj_num[i]   
        obj = []
        obj_num = []
        for i in value:
            if df['P_ID'].value_counts()[i] < 29:
                obj.append(i)
                obj_num.append(29-df['P_ID'].value_counts()[i])

In [23]:
def preprocessing_1(df):

    num1, num2 = num_under(df)

    for i in num1:
        globals()['group{0}'.format(i)] = set(devide(i, df,num2))

    for i in num1:
        globals()['sumnum{}'.format(i)] = rule3_1(df,globals()['group{0}'.format(i)])

    for i in num1:
        globals()['result{}'.format(i)] = rule3_2(i,globals()['sumnum{}'.format(i)], df)
        if globals()['result{}'.format(i)] == None:
            pass
        else:
            globals()['new_start{i}'] = globals()['result{}'.format(i)]

    label_list = []
    for i in num1:
        if globals()['result{}'.format(i)] == 'under':
            label_list.append('group{0}'.format(i))
            label_group(globals()['group{0}'.format(i)],df,i)

    for i in num1:
        if globals()['result{}'.format(i)] == 'under':
            num2 = set(num2) - globals()['group{}'.format(i)]

    num2 = set(num2)
    a = add_label(df, num2)
    rule5(a,df)
    rule6(df)
    rule7(df)

In [24]:
year = [16,17,18,19,20]
mody_label ={}
for i in team:
    for j in year:
        preprocessing_1(globals()['{0}{1}'.format(i,j)])
        print(i,j,mody_label)
        mody_label ={}

LG 16 {8: [3], 9: [3], 11: [3], 10: [3], 6: [1], 4: [5], 7: [5]}
LG 17 {2: [6], 5: [3, 4, 1, 6]}
LG 18 {4: [0], 6: [1], 7: [1], 8: [0], 9: [1], 10: [3, 2, 1]}
LG 19 {9: [7, 2], 10: [7], 11: [7], 5: [2], 6: [2], 8: [2], 3: [7]}
LG 20 {1: [4, 3, 5, 2], 7: [5, 2], 6: [0]}
NC 16 {8: [9], 2: [9], 6: [9], 7: [9], 11: [0], 5: [0, 1, 9, 4, 3], 10: [4, 3]}
NC 17 {6: [0], 7: [0], 8: [1], 9: [0], 10: [2], 3: [1, 4, 0, 5]}
NC 18 {2: [6], 8: [6], 10: [6], 11: [6], 9: [0], 7: [6], 5: [6, 3]}
NC 19 {11: [8, 4], 6: [8, 5], 7: [8, 5], 9: [5], 0: [5], 3: [4], 10: [4, 2, 8]}
NC 20 {4: [5], 6: [5]}
KT 16 {5: [8], 7: [8], 9: [0], 10: [0], 4: [3, 8, 6, 0, 1], 2: [0, 1]}
KT 17 {8: [6], 1: [5], 7: [4], 9: [0], 10: [2], 3: [6], 11: [5, 2, 6]}
KT 18 {8: [1], 9: [1], 10: [1], 11: [1], 4: [0], 7: [0], 5: [6, 1, 2]}
KT 19 {2: [1], 6: [1], 7: [1], 9: [1], 10: [1], 8: [5, 3, 4]}
KT 20 {4: [0], 6: [0]}
SS 16 {2: [8], 12: [8], 6: [8], 7: [8], 11: [0], 9: [0], 5: [10], 1: [3, 10, 8, 4]}
SS 17 {3: [7], 4: [7], 9: [0], 1

In [25]:
year = [16,17,18,19,20]
for i in year:
    for j in team:
        print(i,j)
        ID_label(globals()['{0}{1}'.format(j,i)])
        print()

16 LG
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4}

16 NC
{0: 0, 1: 1, 9: 2, 3: 3, 4: 4}

16 KT
{0: 0, 1: 1, 3: 2, 8: 3, 6: 4}

16 SS
{0: 0, 3: 1, 8: 2, 10: 3, 4: 4}

16 HH
{0: 0, 11: 1, 10: 2, 14: 3, 7: 4}

16 SK
{0: 0, 1: 1, 2: 2, 5: 3, 4: 4}

16 LT
{0: 0, 1: 1, 7: 2, 3: 3, 9: 4}

16 HT
{0: 0, 1: 1, 12: 2, 3: 3, 15: 4}

16 WO
{0: 0, 3: 1, 2: 2, 4: 3, 9: 4}

16 OB
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4}

17 LG
{0: 0, 1: 1, 6: 2, 3: 3, 4: 4}

17 NC
{0: 0, 1: 1, 2: 2, 4: 3, 5: 4}

17 KT
{0: 0, 5: 1, 2: 2, 6: 3, 4: 4}

17 SS
{0: 0, 1: 1, 2: 2, 7: 3, 8: 4}

17 HH
{0: 0, 1: 1, 8: 2, 3: 3, 6: 4}

17 SK
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4}

17 LT
{0: 0, 1: 1, 4: 2, 3: 3, 5: 4}

17 HT
{0: 0, 1: 1, 9: 2, 3: 3, 4: 4}

17 WO
{0: 0, 9: 1, 2: 2, 3: 3, 8: 4}

17 OB
{0: 0, 1: 1, 4: 2, 3: 3, 6: 4}

18 LG
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4}

18 NC
{0: 0, 1: 1, 6: 2, 3: 3, 4: 4}

18 KT
{0: 0, 1: 1, 2: 2, 3: 3, 6: 4}

18 SS
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

18 HH
{0: 0, 1: 1, 2: 2, 5: 3, 4: 4}

18 SK
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

18 LT


In [26]:
for i in year:
    for j in team:
        globals()['ver1_{1}{0}'.format(i,j)]= pd.concat([globals()['{1}{0}'.format(i,j)],
                                             pd.get_dummies(globals()['{1}{0}'.format(i,j)]["P_ID"],
                                                            prefix='P_ID')], axis=1)
        globals()['ver1_{1}{0}'.format(i,j)] = globals()['ver1_{1}{0}'.format(i,j)].drop(columns = ['P_ID'])
        globals()['ver1_{1}{0}'.format(i,j)] = globals()['ver1_{1}{0}'.format(i,j)].drop(columns = ['p_fac'])

In [27]:
for i in team:
    for j in year:
        globals()['ver1_{0}{1}'.format(i,j)].to_csv('idea1/ver1_{0}{1}.csv'.format(i,j), index =False)

In [28]:
def new_train_data(df):
    first = []
    second = []
    third = []
    fourth = []
    for i in range(4,len(df)):
        feat = df['P_ID'][i-4:i].values
        first.append(feat[0])
        second.append(feat[1])
        third.append(feat[2])
        fourth.append(feat[3])
    raw_dat = {'first': first, 'second': second, 'third' : third, 'fourth': fourth}
    data = pd.DataFrame(raw_dat)
#     print(len(data))
#     print(len(df['P_ID'][4:].values))
    data["P_ID"] = df['P_ID'][4:].values
    return data

In [29]:
for i in year:
    for j in team:
        globals()['new_{0}{1}'.format(j,i)] = new_train_data(globals()['{0}{1}'.format(j,i)])
        globals()['new_{0}{1}'.format(j,i)].to_csv('idea2/new_{0}{1}.csv'.format(j,i), index= False)

In [30]:
def read_ver3(what, how):
#원하는 데이터 연도별로 다 불러오가
#what은 무슨 기록인지 받는 입력   예)팀타자, 개인투수....
#how는 생성할 데이터 프레임 이름  예)df_raw, df_bat...
    year = [16,17,18,19,20]
    for i in year:
        globals()['ver1_{0}{1}'.format(how,i)] = pd.read_csv("idea1/ver1_{0}{1}.csv".format(what,i))
        globals()['ver2_{0}{1}'.format(how,i)] = pd.read_csv("idea2/new_{0}{1}.csv".format(what,i))

In [31]:
for i in team:
    read_ver3(i,i)

In [32]:
ver2_HH20

,first,second,third,fourth,P_ID
0,0,1,1,2,0
1,1,1,2,0,0
2,1,2,0,0,3
3,2,0,0,3,1
4,0,0,3,1,2
...,...,...,...,...,...
56,3,4,1,0,0
57,4,1,0,0,3
58,1,0,0,3,4
59,0,0,3,4,1


In [33]:
ver1_col = list(ver1_HH16.columns)
ver2_col = list(ver2_HH16.columns)

In [34]:
for i in team:
    for j in year:
        globals()['{0}{1}'.format(i,j)] = pd.concat([globals()['ver2_{0}{1}'.format(i,j)][ver2_col[:-1]],
                                                               globals()['ver1_{0}{1}'.format(i,j)][ver1_col[3:]].loc[4:].reset_index(drop=True) ] ,axis = 1)

In [35]:
# df = pd.concat([ver2_HH16[ver2_col[:-1]], ver1_HH16[ver1_col[3:]].loc[4:].reset_index(drop=True) ] ,axis = 1)
# team = ['HH', 'SS', 'NC', 'LG', 'SK', 'LT', 'OB', 'HT', 'KT', 'WO']
df = pd.concat([HH16,SS16,NC16,LG16,SK16,LT16,OB16,HT16, KT16, WO16,
                  HH17,SS17,NC17,LG17,SK17,LT17,OB17,HT17, KT17, WO17,
                  HH18,SS18,NC18,LG18,SK18,LT18,OB18,HT18, KT18, WO18,
                  HH19,SS19,NC19,LG19,SK19,LT19,OB19,HT19, KT19, WO19,
                  HH20,SS20,NC20,LG20,SK20,LT20,OB20,HT20, KT20, WO20
               ], axis=0)

In [36]:
def pre_data(df):
    df= pd.concat([df, pd.get_dummies(df["VS_T_ID"],prefix='VS_T_ID')],axis=1)
    df = df.drop(columns = ['VS_T_ID'])
    df = df.reset_index(drop= True)
    return df

In [37]:
df = pre_data(df)

In [38]:
def drop(df):
    drop_index=[]
    for i in range(len(df)):
        li = list(df.loc[i,'first':'fourth'].values)
        for j in range(3):
            if li[j] == li[j+1]:
                drop_index.append(i)
                break
    for k in drop_index:
        df = df.drop(k)
    df = df.reset_index(drop= True)
    return df

In [39]:
df = drop(df)

In [40]:
label_cols= list(df.columns)[4:9]
feature_cols = list(df.columns)[:4] # +list(df.columns)[9:]
# 데이터를 묶어서 처리하는 함수
def make_dateset(data, label, window_size=10):
    import numpy as np
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size+1]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [41]:
train = df[:]
train_feature, train_label = make_dateset(train[feature_cols], train[label_cols],5)

In [42]:
VALID_SIZE = 355
x_train = train_feature[:-VALID_SIZE]
x_valid = train_feature[VALID_SIZE:]
y_train = train_label[:-VALID_SIZE]
y_valid = train_label[VALID_SIZE:]

In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [44]:
def stacked_lstm(n_neurons=32):
    model = Sequential()
    model.add(LSTM(n_neurons, input_shape =  [train_feature.shape[1], train_feature.shape[2]], return_sequences = True, activation ='relu'))
    model.add(LSTM(n_neurons, return_sequences = False, activation ='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation ='softmax'))
   
    # adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [45]:
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
    "n_neurons": np.arange(1, 100),
}

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10)

with tf.device('/GPU:0'):
    keras_reg = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = stacked_lstm)
    rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
    rnd_search_cv.fit(x_train, y_train, epochs=500, validation_data=(x_valid, y_valid),
                      callbacks=[early_stop])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_neurons=90 ....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 3333 samples, validate on 5000 samples
Epoch 1/500
3333/3333 [==============================] - 9s 3ms/sample - loss: 1.6045 - accuracy: 0.2241 - val_loss: 1.5787 - val_accuracy: 0.2174
Epoch 2/500
3333/3333 [==============================] - 4s 1ms/sample - loss: 1.5117 - accuracy: 0.3141 - val_loss: 1.4300 - val_accuracy: 0.3808
Epoch 3/500
3333/3333 [==============================] - 4s 1ms/sample - loss: 1.4148 - accuracy: 0.3780 - val_loss: 1.3642 - val_accuracy: 0.4578
Epoch 4/500
3333/3333 [==============================] - 4s 1ms/sample - loss: 1.3575 - accuracy: 0.4329 - val_loss: 1.3136 - val_accuracy: 0.5070
Epoch 5/500
3333/3333 [==============================] - 4s 1ms/sample - loss: 1.3110 - accuracy: 0.4602 - val_loss: 1.2754 - val_accuracy: 0.5332
Epoch 6/500
3333/3333 [==============================] - 5s 2ms/sample - loss: 1.2673 - accuracy: 0.5146 - val_loss: 1.2377 - val_accuracy: 0.5890
Epoch 7/500
3333/3333 [==============================] - 5s 1ms/sample

Epoch 56/500
3333/3333 [==============================] - 5s 1ms/sample - loss: 0.6558 - accuracy: 0.7699 - val_loss: 0.8425 - val_accuracy: 0.7454
Epoch 57/500
3333/3333 [==============================] - 3s 1ms/sample - loss: 0.6644 - accuracy: 0.7762 - val_loss: 0.8402 - val_accuracy: 0.7526
Epoch 58/500
1667/1667 [==============================] - 0s 255us/sample - loss: 1.6393 - accuracy: 0.5609
[CV] ..................................... n_neurons=90, total= 4.0min
[CV] n_neurons=90 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.0min remaining:    0.0s


...

Train on 5000 samples, validate on 5000 samples
Epoch 1/500
5000/5000 [==============================] - 11s 2ms/sample - loss: 1.6034 - accuracy: 0.2362 - val_loss: 1.5636 - val_accuracy: 0.2980
Epoch 2/500
5000/5000 [==============================] - 5s 942us/sample - loss: 1.4958 - accuracy: 0.3188 - val_loss: 1.3998 - val_accuracy: 0.4024
Epoch 3/500
5000/5000 [==============================] - 5s 918us/sample - loss: 1.3972 - accuracy: 0.3934 - val_loss: 1.3233 - val_accuracy: 0.4752
Epoch 4/500
5000/5000 [==============================] - 5s 955us/sample - loss: 1.3596 - accuracy: 0.4264 - val_loss: 1.2861 - val_accuracy: 0.4832
Epoch 5/500
5000/5000 [==============================] - 5s 935us/sample - loss: 1.3370 - accuracy: 0.4418 - val_loss: 1.2504 - val_accuracy: 0.5282
Epoch 6/500
5000/5000 [==============================] - 5s 971us/sample - loss: 1.3105 - accuracy: 0.4716 - val_loss: 1.2226 - val_accuracy: 0.5442
Epoch 7/500
5000/5000 [==============================] - 5s

5000/5000 [==============================] - 4s 777us/sample - loss: 0.9540 - accuracy: 0.6616 - val_loss: 0.8421 - val_accuracy: 0.7056
Epoch 56/500
5000/5000 [==============================] - 4s 767us/sample - loss: 0.9358 - accuracy: 0.6652 - val_loss: 0.8413 - val_accuracy: 0.7066
Epoch 57/500
5000/5000 [==============================] - 4s 794us/sample - loss: 0.9452 - accuracy: 0.6588 - val_loss: 0.8475 - val_accuracy: 0.7040
Epoch 58/500
5000/5000 [==============================] - 4s 837us/sample - loss: 0.9332 - accuracy: 0.6692 - val_loss: 0.8236 - val_accuracy: 0.7062
Epoch 59/500
5000/5000 [==============================] - 4s 877us/sample - loss: 0.9377 - accuracy: 0.6654 - val_loss: 0.8309 - val_accuracy: 0.7104
Epoch 60/500
5000/5000 [==============================] - 5s 944us/sample - loss: 0.9403 - accuracy: 0.6658 - val_loss: 0.8099 - val_accuracy: 0.7098
Epoch 61/500
5000/5000 [==============================] - 4s 759us/sample - loss: 0.9189 - accuracy: 0.6686 - val

Epoch 110/500
5000/5000 [==============================] - 5s 943us/sample - loss: 0.7875 - accuracy: 0.7078 - val_loss: 0.6728 - val_accuracy: 0.7576
Epoch 111/500
5000/5000 [==============================] - 5s 984us/sample - loss: 0.7842 - accuracy: 0.7160 - val_loss: 0.6683 - val_accuracy: 0.7642
Epoch 112/500
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.7880 - accuracy: 0.7070 - val_loss: 0.6617 - val_accuracy: 0.7592
Epoch 113/500
5000/5000 [==============================] - 5s 966us/sample - loss: 0.7676 - accuracy: 0.7202 - val_loss: 0.6449 - val_accuracy: 0.7686
Epoch 114/500
5000/5000 [==============================] - 5s 911us/sample - loss: 0.7712 - accuracy: 0.7116 - val_loss: 0.6433 - val_accuracy: 0.7630
Epoch 115/500
5000/5000 [==============================] - 5s 905us/sample - loss: 0.7789 - accuracy: 0.7068 - val_loss: 0.6354 - val_accuracy: 0.7662
Epoch 116/500
5000/5000 [==============================] - 5s 926us/sample - loss: 0.7670 - accu

5000/5000 [==============================] - 9s 2ms/sample - loss: 0.6595 - accuracy: 0.7442 - val_loss: 0.5350 - val_accuracy: 0.8058
Epoch 165/500
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.6554 - accuracy: 0.7488 - val_loss: 0.5431 - val_accuracy: 0.8074
Epoch 166/500
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.6596 - accuracy: 0.7484 - val_loss: 0.5316 - val_accuracy: 0.8144
Epoch 167/500
5000/5000 [==============================] - 5s 911us/sample - loss: 0.6473 - accuracy: 0.7496 - val_loss: 0.5289 - val_accuracy: 0.8110
Epoch 168/500
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.6471 - accuracy: 0.7514 - val_loss: 0.5218 - val_accuracy: 0.8106
Epoch 169/500
5000/5000 [==============================] - 5s 953us/sample - loss: 0.6533 - accuracy: 0.7520 - val_loss: 0.5227 - val_accuracy: 0.8174
Epoch 170/500
5000/5000 [==============================] - 5s 967us/sample - loss: 0.6549 - accuracy: 0.7452 - val_l

5000/5000 [==============================] - 5s 991us/sample - loss: 0.5633 - accuracy: 0.7840 - val_loss: 0.4574 - val_accuracy: 0.8390
Epoch 219/500
5000/5000 [==============================] - 5s 903us/sample - loss: 0.5752 - accuracy: 0.7820 - val_loss: 0.4809 - val_accuracy: 0.8290
Epoch 220/500
5000/5000 [==============================] - 4s 887us/sample - loss: 0.5786 - accuracy: 0.7732 - val_loss: 0.4824 - val_accuracy: 0.8316
Epoch 221/500
5000/5000 [==============================] - 5s 940us/sample - loss: 0.5867 - accuracy: 0.7722 - val_loss: 0.4714 - val_accuracy: 0.8406
Epoch 222/500
5000/5000 [==============================] - 5s 968us/sample - loss: 0.5643 - accuracy: 0.7788 - val_loss: 0.4686 - val_accuracy: 0.8392
Epoch 223/500
5000/5000 [==============================] - 5s 928us/sample - loss: 0.5686 - accuracy: 0.7794 - val_loss: 0.4690 - val_accuracy: 0.8368
Epoch 224/500
5000/5000 [==============================] - 5s 933us/sample - loss: 0.5579 - accuracy: 0.7904

In [46]:
print('rnd_search_cv.best_params_:', rnd_search_cv.best_params_)
print('rnd_search_cv.best_score_:', rnd_search_cv.best_score_)

rnd_search_cv.best_params_: {'n_neurons': 35}
rnd_search_cv.best_score_: 0.6098000058650971


In [47]:
# first_list는 첫 경기를 예측하기 위한 feature이다.
# org_test_feature는 20년 팀의 test_feature이다.
def gen_pred(first_list, org_test_feature):
    gen_num = 139- len(org_test_feature)
    for i in range(gen_num):
        new_list = np.array([first_list])
        a = org_test_feature[-1][1:]
        pred_team = np.concatenate((a, new_list), axis=0)
        org_test_feature = np.concatenate((org_test_feature,np.array([pred_team])), axis=0)
        next_start=model.predict(org_test_feature)
        b = first_list[1:]
        b.append(np.argmax(next_start[-1]))
        first_list = b
    return(org_test_feature)

In [48]:
# 아래는 first_list들이다.
HH = [3,4,1,0]
SS = [4,2,0,1]
NC = [0,1,2,3]
LG = [2,4,0,1]
SK = [1,4,2,3]
LT = [0,1,2,0]
OB = [2,3,4,1]
HT = [0,1,4,2]
KT = [1,3,0,0]
WO = [0,3,4,0]
col = ['first', 'second', 'third', 'fourth','fifth']

In [50]:
model = rnd_search_cv.best_estimator_.model

In [51]:
test =  HH20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(HH,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/HH_predict.csv', index=False)

In [52]:
test =  SS20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(SS,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/SS_predict.csv', index=False)

In [53]:
test =  NC20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(NC,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/NC_predict.csv', index=False)

In [54]:
test =  LG20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(LG,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/LG_predict.csv', index=False)

In [55]:
test =  SK20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(SK,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/SK_predict.csv', index=False)

In [56]:
test =  LT20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(LT,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/LT_predict.csv', index=False)

In [57]:
test =  OB20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(OB,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/OB_predict.csv', index=False)

In [58]:
test =  HT20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(HT,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/HT_predict.csv', index=False)

In [59]:
test =  KT20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(KT,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/KT_predict.csv', index=False)

In [60]:
test =  WO20 [:]
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)
test_feature = gen_pred(WO,test_feature)
pred = model.predict(test_feature)
pred = pd.DataFrame(data=pred, columns = col)
pred.to_csv('pred/WO_predict.csv', index=False)